In [41]:
## pdf 라이브러리
# import aspose.pdf as ap
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

import pandas as pd
from glob import glob
import os
from tqdm import tqdm
import zipfile

In [42]:
def local_folder_make(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [43]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname=relative_path)

In [44]:
path = "../data/juju_data_add_20150101_20230919.csv"
df = pd.read_csv(path, encoding="utf-8-sig")


In [45]:
columns_mean = {"CST_CNT":"보통주식(주)의 가격",
                "PST_CNT":"기타주식(주)의 가격",
                "CST_ISS_VAL":"신주 발행가액에서 확정발행가의 보통주식(주)의 가격",
                "PST_ISS_VAL":"신주 발행가액에서 확정발행가의 기타주식(주)의 가격",
                "PRE_CST_VAL":"신주 발행가액에서 예정발행가의 보통주식(주)의 가격",
                "PRE_PST_VAL":"신주 발행가액에서 예정발행가의 기타주식(주)의 가격",
                "PRE_CST_VAL_DT":"신주 발행가액에서 예정발행가의 보통주식(주)의 확정예정일",
                "PRE_PST_VAL_DT":"신주 발행가액에서 예정발행가의 기타주식(주)의 확정예정일",
                "ALL_BS_DT":"신주배정기준일",
                "NEW_ASN_CNT":"1주당 신주배정주식수 (주)",
                "EMP_BGN_DT":"우리사주조합 시작일",
                "EMP_END_DT":"우리사주조합 종료일",
                "SH_BGN_DT":"구주주 시작일",
                "SH_END_DT":"구주주 종료일",
                "PYM_DT":"납입일",
                "PRC_PLN":"실권주 처리계획",
                "LST_PLN_DT":"신주의 상장예정일",
                "CHF_AGN":"대표주관회사",
                "NST_GV_YN":"신주인수권 양도여부",
                "FIP_CM":"확정 발행가액 산정방법",
                "NST_RM":"신주인수권에 관한 사항",
                "NST_SCH_DT":"신주인수권 거래소 거래기간(신주인수권증서 상장 예정기간)",
                "CI_MTH":"증자방식",
                "NOTES":"기타 투자판단에 참고할 사항",
                "PUB_ANN_DT":"공시날짜",}

In [46]:
# hangle_font_path = "C:/Windows/Fonts/NanumGothic.ttf"
# def create_pdf(series, save_path):
#     RCEPT_NO = series.RCEPT_NO
#     PUB_ANN_DT = series.PUB_ANN_DT
    
#     series = dict(series)
#     result = ["%s : %s"%(columns_mean[key], value) for key, value in series.items() if key in columns_mean]

#     # 문서 개체 초기화
#     document = ap.Document()
#     # 페이지 추가
#     page = document.pages.add()
    
#     for i in result:
#         # textfragment 객체 초기화
#         text_fragment = ap.text.TextFragment(i)
        
#         # 폰트 설정
#         text_fragment.text_state.font_size = 12  # 원하는 폰트 크기로 설정
#         # 폰트 파일 로드 및 적용
#         font = ap.text.FontRepository.open_font(hangle_font_path)
#         text_fragment.text_state.font = font

#         # 새 페이지에 텍스트 조각 추가
#         page.paragraphs.add(text_fragment)


#     local_path = os.path.join(save_path, PUB_ANN_DT)
#     local_folder_make(local_path)
#     file_path = os.path.join(local_path, str(RCEPT_NO)+".pdf")
    
#     # 업데이트된 PDF 저장
#     document.save(file_path)
    
    
def create_pdf(series, file_path):
    series = dict(series)
    result = ["● %s : %s"%(columns_mean[key], value) for key, value in series.items() if key in columns_mean]
    
    # Create a SimpleDocTemplate and add some content to it.
    doc = SimpleDocTemplate(file_path, pagesize=letter)

    # Get the default style set.
    styles = getSampleStyleSheet()

    # Override the default font size and name.
    styles['Normal'].fontName = 'Malgun'
    styles['Normal'].fontSize = 10
    styles['Normal'].leading = 20

    # Create a list to hold the content that will be added to the document.
    content = []

    # Add some text with automatic line breaks.
    # long_text = "안녕하세요 " * 500   # This will be a very long line of text.
    for text in result:
        paragraph = Paragraph(text, styles['Normal'])
        content.append(paragraph)

    # Build the PDF document with the content list.
    doc.build(content)
    


In [47]:
pdf_path = "../data/pdf"
for i in tqdm(range(len(df))):
    series = df.iloc[i]
    RCEPT_NO = series.RCEPT_NO
    PUB_ANN_DT = series.PUB_ANN_DT
    
    local_path = os.path.join(pdf_path, PUB_ANN_DT)
    local_folder_make(local_path)
    file_path = os.path.join(local_path, str(RCEPT_NO)+".pdf")
    
    create_pdf(series, file_path)
    

100%|██████████| 84/84 [00:02<00:00, 35.34it/s]


In [48]:
for path in glob(pdf_path+"/*"):
    # 폴더 압축 실행
    zip_folder(path, path+".zip")